# 导库

In [1]:
# 图像采集和处理库
import cv2    #OpenCV库
import imutils #配合OpenCV
import numpy as np
from keras.preprocessing.image import img_to_array # 图片转化成数组

# 加载模型所需库
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.layers import Activation

Using TensorFlow backend.


In [2]:
# 加载数据和图像的参数
emotion_model_path = '../Models/改进模型2.h5' # 预训练模型
detection_model_path = '../Haarcascade/haarcascade_frontalface_default.xml' # 人脸Haar特征分类器
EMOTIONS = ["Angry" ,"Disgust","Fear", "Happy", "Sad", "Surprise","Neutral"] # 情绪类别

# 加载模型、建立人脸检测器
face_detection = cv2.CascadeClassifier(detection_model_path) # 人脸检测的一个级联分类器
emotion_classifier = load_model(emotion_model_path, compile=False) # 加载预训练模型

# 实时的人脸情绪识别

In [ ]:
# 打开笔记本摄像头
video_capture = cv2.VideoCapture(0)

while True:
    # 逐帧捕捉
    ret, frame = video_capture.read()

    i = 0 # 人脸计数
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 颜色空间转换，灰度图
    
    # 检测出图片中所有的人脸，并将人脸用faces保存各个人脸的坐标、大小（用矩形表示），函数由分类器对象调用
    
    faces = face_detection.detectMultiScale(gray,               # 捕获的图像经过灰度处理后的数据
                                            scaleFactor=1.3,    # 每次图像尺寸减小的比例
                                            minNeighbors=5,     # 每一个目标至少要被检测到5次才算是真的目标
                                            minSize=(48,48),    # 目标的最小尺寸
                                            flags=cv2.CASCADE_SCALE_IMAGE) # 函数不进行分类器缩放，而是缩放图像进行检测

#     if len(faces) > 0:
#         faces = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0] 

    # 多人脸识别
    for (x, y, w, h) in faces:
        #人脸坐标：x,y，矩形宽高：w，h
#             (x, y, w, h) = faces

        #从灰度图像中提取人脸的面部关键点，将其调整为固定的48*48像素（预训练模型形状）
        facial = gray[y:y + h, x:x + w]
        facial = cv2.resize(facial, (48, 48))
        facial = facial.astype("float") / 255.0
        facial = img_to_array(facial, data_format="channels_first")#data_format="channels_first"
        facial = np.expand_dims(facial, axis=0)
#         print(facial)

        # 预测情绪
        prediction = emotion_classifier.predict(facial)
        prediction_result = np.argmax(prediction)

        # 人脸识别框
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        cv2.putText(frame, "Face {}".format(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 添加预测概率
        cv2.putText(frame, "----------------",(40,60 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Emotional report : Face " + str(i+1),(40,80 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Angry : " + str(round(prediction[0][0],3)),(40,100 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Disgust : " + str(round(prediction[0][1],3)),(40,120 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Fear : " + str(round(prediction[0][2],3)),(40,140 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Happy : " + str(round(prediction[0][3],3)),(40,160 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Sad : " + str(round(prediction[0][4],3)),(40,180 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Surprise : " + str(round(prediction[0][5],3)),(40,200 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(frame, "Neutral : " + str(round(prediction[0][6],3)),(40,220 + 180*i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # 用标签注释主图像
        if prediction_result == 0 :
            cv2.putText(frame, "Angry",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 1 :
            cv2.putText(frame, "Disgust",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 2 :
            cv2.putText(frame, "Fear",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 3 :
            cv2.putText(frame, "Happy",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 4 :
            cv2.putText(frame, "Sad",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 5 :
            cv2.putText(frame, "Surprise",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else :
            cv2.putText(frame, "Neutral",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
        i = i+1
    # 检测到的人脸数
    cv2.putText(frame,'Number of Faces : ' + str(len(faces)),(40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
    
#     cv2.namedWindow("Video", 0)
#     cv2.resizeWindow("Video", 1000, 1000)  # 设置窗口大小
    
    cv2.imshow('Video', frame)
    
    # 100ms捕捉一次，按q退出捕捉
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

# 结束后，释放捕获
video_capture.release()
cv2.destroyAllWindows()